# Webcraping Data and Make DF of Postal Code
This notebook webscrapes data from this [wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), which contains lists of postal code of Canada. Then it makes a dataframe and saves it.  
There are following instructions on the dataframe:
1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [2]:
#install BeautifulSoup if not yet done
#!pip install BeautifulSoup4

#install lxml parser package if not yet done
#!pip install lxml

#install requests library if not yet done
#!pip install requests

#importing libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
#define variabels
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
#get sourcode of wiki page
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#make soup
soup = BeautifulSoup(source.text,'lxml')
#find the postal code table
tb = soup.table
#find all rows
t_rows = tb.find_all('tr')

In [9]:
#convert into dataframe
df = pd.DataFrame()
for tr in t_rows:
    td = tr.find_all('td')
    rows = [i.text for i in td]
    if pd.Series(rows).empty == False:
        df =df.append(pd.Series(rows),ignore_index=True)

#clean dataframe
col_dic = dict(zip(df.columns,['PostalCode','Borough','Neighborhood']))
df = df.rename(columns = col_dic ).apply(lambda x: x.str.rstrip('\n'))
df = df[df['Borough'] !='Not assigned'].reset_index().drop(columns='index')
df['Neighborhood'] = df['Neighborhood'].apply(lambda x: x.replace(' /',','))

#save dataframe to .csv file
df.to_csv('postal_code')

df

/Users/mr.x/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
..         ...               ...   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4          Queen's Park, Ontario Provincial Government  
..                                                 ...  
98      The Kingsway, Montgomery Road , Old Mill North  
99                                Church and Wellesley  
100              Business reply mail Processing CentrE  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  
102  Mimico NW, The Queensway West, South of Bloor,...  

[103 rows x 3 columns]

In [10]:
df.shape

(103, 3)